In [1]:
import sklearn
print('Hi')
print(sklearn.__version__)

Hi
0.24.2


## Create Modells

In [1]:
import pandas as pd
import sklearn as sk
import numpy as np
import pickle

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

from sklearn.neural_network import MLPRegressor

from autosklearn.regression import AutoSklearnRegressor
import autosklearn.metrics

from pathlib import Path

import time # localtime to unix time

noise = 0
folderinput = "../workdir/AIS-ModelFrames/"
folderoutput = "../workdir/AIS-TrainedModel/"

filename = "aisdk-2023-11-08_1_kkn"

for known in range(12, 2, -1):

    train = pd.read_csv(folderinput+filename+"_train_"+str(known)+"_0.csv")
    test = pd.read_csv(folderinput+filename+"_test_"+str(known)+"_0.csv")

    train_y = train.loc[:,["output_x","output_y"]]
    test_y = test.loc[:,["output_x","output_y"]]
    train_x = train.drop(columns=["output_x","output_y"])
    test_x = test.drop(columns=["output_x","output_y"])

    test = []
    train = []

    df_res = test_y.copy()

    df_res.rename(columns={"output_x": "known_"+str(known)+"_0", "output_y": "noise_"+str(known)+"_0"}, inplace=True)

    df_res[f"known_"+str(known)+"_0"] = known
    df_res[f"noise_"+str(known)+"_0"] = noise
    print("init "+str(known)+" "+str(noise) + " " + time.ctime(time.time())) 

    # linear regression

    path = Path(folderoutput+filename+"_lin_"+str(known)+"_0.csv")
    if not path.exists():  

        linearpipe = Pipeline([("scaler", StandardScaler()), ("regressor", LinearRegression())])
        linearpipe.fit(train_x, train_y)
        pickle.dump(linearpipe, open(path, 'wb'))
        linearpipe = []
    print("lin "+str(known)+" "+str(noise) + " " + time.ctime(time.time())) 

    # Multi-layer Perceptron

    path = Path(folderoutput+filename+"_mlp_"+str(known)+"_0.csv")
    if not path.exists(): 

        mlppipe = Pipeline([("scaler", StandardScaler()), ("regressor", MLPRegressor((50)))])
        mlppipe.fit(train_x, train_y)
        pickle.dump(mlppipe, open(path, 'wb'))
        mlppipe = []
    print("mlp "+str(known)+" "+str(noise) + " " + time.ctime(time.time())) 

    # autosklearn

    runduraction = 3600*5#2
    memory = 10*1024
    path = Path(folderoutput+filename+"_autosk_"+str(known)+"_0_"+str(runduraction)+"_"+str(memory)+".csv")

    if not path.exists(): 
        automl = AutoSklearnRegressor(
            time_left_for_this_task=runduraction, memory_limit=memory, n_jobs=-1, metric=autosklearn.metrics.mean_squared_error
            #time_left_for_this_task=3600, memory_limit=80*1024, n_jobs=-1, metric=autosklearn.metrics.mean_squared_error
            )
        automl.fit(train_x, train_y, dataset_name=f'known{known}')

        ensemble_dict = automl.show_models()
        print(ensemble_dict)
        pickle.dump(automl, open(path, 'wb'))
        automl = []


     
    print("sklearn "+str(known)+" "+str(noise) + " " + time.ctime(time.time()))   

init 12 0 Fri Jan 24 11:24:55 2025
lin 12 0 Fri Jan 24 11:24:55 2025
mlp 12 0 Fri Jan 24 11:24:55 2025
[WARNING] [2025-01-24 11:25:09,549:Client-AutoML(1):known12] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (17995.410707)
[WARNING] [2025-01-24 11:25:09,549:Client-AutoML(1):known12] Capping the per_run_time_limit to 8997.0 to have time for a least 2 models in each process.
[WARNING] [2025-01-24 11:25:09,984:Client-AutoMLSMBO(1)::known12] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2025-01-24 11:25:21,932:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-24 11:25:26,357:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-24 11:25:34,648:Client-EnsembleBuilder] No ru

/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


mlp 11 0 Fri Jan 24 16:53:38 2025
[WARNING] [2025-01-24 16:53:49,985:Client-AutoML(1):known11] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (17996.094015)
[WARNING] [2025-01-24 16:53:49,985:Client-AutoML(1):known11] Capping the per_run_time_limit to 8998.0 to have time for a least 2 models in each process.
[WARNING] [2025-01-24 16:53:50,547:Client-AutoMLSMBO(1)::known11] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2025-01-24 16:54:01,296:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-24 16:54:05,356:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-24 16:54:18,253:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-24 16:

/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


mlp 10 0 Fri Jan 24 22:22:06 2025
[WARNING] [2025-01-24 22:23:32,328:Client-AutoML(1):known10] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (17996.297691)
[WARNING] [2025-01-24 22:23:32,328:Client-AutoML(1):known10] Capping the per_run_time_limit to 8998.0 to have time for a least 2 models in each process.
[WARNING] [2025-01-24 22:23:32,857:Client-AutoMLSMBO(1)::known10] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2025-01-24 22:23:43,552:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-24 22:23:47,425:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-24 22:23:53,711:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-24 22:

...

mlp 5 0 Sun Jan 26 00:54:05 2025
[WARNING] [2025-01-26 00:54:16,578:Client-AutoML(1):known5] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (17997.402700)
[WARNING] [2025-01-26 00:54:16,578:Client-AutoML(1):known5] Capping the per_run_time_limit to 8998.0 to have time for a least 2 models in each process.
[WARNING] [2025-01-26 00:54:16,787:Client-AutoMLSMBO(1)::known5] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2025-01-26 00:54:23,261:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-26 00:54:27,965:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-26 00:54:33,745:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-26 00:54:4

/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


mlp 4 0 Sun Jan 26 06:07:24 2025
[WARNING] [2025-01-26 06:07:41,531:Client-AutoML(1):known4] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (17997.684838)
[WARNING] [2025-01-26 06:07:41,531:Client-AutoML(1):known4] Capping the per_run_time_limit to 8998.0 to have time for a least 2 models in each process.
[WARNING] [2025-01-26 06:07:41,741:Client-AutoMLSMBO(1)::known4] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2025-01-26 06:07:47,431:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-26 06:07:51,571:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-26 06:07:55,402:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-26 06:08:0

/home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


mlp 3 0 Sun Jan 26 11:20:13 2025
[WARNING] [2025-01-26 11:20:26,577:Client-AutoML(1):known3] Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (17997.949577)
[WARNING] [2025-01-26 11:20:26,577:Client-AutoML(1):known3] Capping the per_run_time_limit to 8998.0 to have time for a least 2 models in each process.
[WARNING] [2025-01-26 11:20:26,761:Client-AutoMLSMBO(1)::known3] Could not find meta-data directory /home/sebastian/Dokumente/Python-Git/py-ma-git/venv/lib/python3.9/site-packages/autosklearn/metalearning/files/mean_squared_error_multioutput.regression_dense
[WARNING] [2025-01-26 11:20:32,293:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-26 11:20:35,339:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-26 11:20:39,137:Client-EnsembleBuilder] No runs were available to build an ensemble from
[WARNING] [2025-01-26 11:20:4

## Check Modells under noise

In [3]:
import numpy as np
import pandas as pd

folder = "../workdir/AIS-ModelFrames/"
filename = f"aisdk-2023-11-08_1_kkn"


for known in range(3, 13, 1):
    for noise in range(0, 11, 5):
        test = pd.read_csv(f"{folder}{filename}_test_{known}_{noise}.csv")
        test_y = test.loc[:,["output_x","output_y"]]
        test_x = test.drop(columns=["output_x","output_y"])

        # Baseline
        dummy_prediction = test_x.loc[:,[f"input_{known-1}_x", f"input_{known-1}_y"]]
        df_res[f"bl_{known}w{noise}"] = np.sqrt((test_y["output_x"] - dummy_prediction[f"input_{known-1}_x"])**2 + (test_y["output_y"] - dummy_prediction[f"input_{known-1}_y"])**2)

        # Linear Regression
        linReg = pickle.load(open(f"../workdir/AIS-TrainedModel/{filename}_lin_{known}_0.csv", 'rb'))
        df = pd.DataFrame(linReg.predict(test_x))
        df.columns = [f"lin_x_{known}_{noise}", f"lin_y_{known}_{noise}"]
        df_res[f"lr_{known}w{noise}"] = np.sqrt((test_y["output_x"] - df[f"lin_x_{known}_{noise}"])**2 + (test_y["output_y"] - df[f"lin_y_{known}_{noise}"])**2)

        # Neural Network
        mlppipe = pickle.load(open(f"../workdir/AIS-TrainedModel/{filename}_mlp_{known}_0.csv", 'rb'))
        df = pd.DataFrame(mlppipe.predict(test_x))
        df.columns = [f"mlp_x_{known}_{noise}", f"mlp_y_{known}_{noise}"]
        df_res[f"ml_{known}w{noise}"] = np.sqrt((test_y["output_x"] - df[f"mlp_x_{known}_{noise}"])**2 + (test_y["output_y"] - df[f"mlp_y_{known}_{noise}"])**2)
        
        # AutoSklearn
        automl = pickle.load(open(f"../workdir/AIS-TrainedModel/{filename}_autosk_{known}_0_18000_10240.csv", 'rb'))
        df = pd.DataFrame(automl.predict(test_x))
        df.columns = [f"autosk_x_{known}_{noise}", f"autosk_y_{known}_{noise}"]
        df_res[f"as_{known}w{noise}"] = np.sqrt((test_y["output_x"] - df[f"autosk_x_{known}_{noise}"])**2 + (test_y["output_y"] - df[f"autosk_y_{known}_{noise}"])**2)
        

        df_tmp = df_res.copy()
        df_res = []
        df_res = df_tmp.copy()
        df_tmp = []

        df_tmp = df_res.copy()
        df_res = df_tmp.copy()

        print(f"Known: {known}, Noise: {noise}")   
         
df_res.to_csv(f"../workdir/AIS-TrainedModel/{filename}_result_18000_10240.csv", index=False)

Known: 3, Noise: 0
Known: 3, Noise: 5
Known: 3, Noise: 10
Known: 4, Noise: 0
Known: 4, Noise: 5
Known: 4, Noise: 10
Known: 5, Noise: 0
Known: 5, Noise: 5
Known: 5, Noise: 10
Known: 6, Noise: 0
Known: 6, Noise: 5
Known: 6, Noise: 10
Known: 7, Noise: 0
Known: 7, Noise: 5
Known: 7, Noise: 10
Known: 8, Noise: 0
Known: 8, Noise: 5
Known: 8, Noise: 10
Known: 9, Noise: 0
Known: 9, Noise: 5
Known: 9, Noise: 10
Known: 10, Noise: 0
Known: 10, Noise: 5
Known: 10, Noise: 10
Known: 11, Noise: 0
Known: 11, Noise: 5
Known: 11, Noise: 10
Known: 12, Noise: 0
Known: 12, Noise: 5
Known: 12, Noise: 10


,known_3_0,noise_3_0,bl_3w0,lr_3w0,ml_3w0,as_3w0,bl_3w5,lr_3w5,ml_3w5,as_3w5,...,ml_12w0,as_12w0,bl_12w5,lr_12w5,ml_12w5,as_12w5,bl_12w10,lr_12w10,ml_12w10,as_12w10
0,3,0,57.472863,10.140397,0.763702,2.222782,61.811366,17.545444,8.268143,12.449421,...,0.735245,10.088977,9.830894,10.521388,11.097213,10.426805,2.271099,17.938708,17.271217,30.833876
1,3,0,43.796119,41.747681,6.726233,6.545459,43.533252,39.969210,5.161478,6.205713,...,2.131380,10.369497,8.190563,14.116897,7.758233,3.328687,17.641131,10.797802,5.225336,13.942684
2,3,0,13.619471,1.655019,1.581207,0.935977,14.102834,2.413462,1.627959,1.062300,...,4.617921,8.597274,24.761415,6.543393,5.321836,5.777006,45.420167,19.468775,28.649034,19.515615
3,3,0,55.693806,34.990134,9.112156,5.497997,54.790430,36.701537,15.937116,15.933388,...,4.760408,12.912032,37.637027,2.737848,10.117210,2.153534,38.770557,19.232818,20.951631,25.858540


## Analysis

In [5]:
import pandas as pd

folder = "../workdir/AIS-TrainedModel/"
filename = f"aisdk-2023-11-08_1_kkn"

df_res = pd.read_csv(f"{folder}{filename}_result_18000_10240.csv")

for known in range(3, 13, 1):
    for noise in range(0, 11, 5):
        for model in ["bl", "lr", "ml", "as"]:
            print(f"{model}_{known}w{noise}")
            print(df_res[f"{model}_{known}w{noise}"].describe())

bl_3w0
count    1.112344e+06
mean     2.909283e+01
std      2.580962e+01
min      0.000000e+00
25%      4.638965e+00
50%      2.269097e+01
75%      4.984155e+01
max      2.999323e+02
Name: bl_3w0, dtype: float64
lr_3w0
count    1.112344e+06
mean     8.837056e+00
std      1.310551e+01
min      7.363258e-04
25%      1.428050e+00
50%      3.766015e+00
75%      1.011780e+01
max      2.624741e+02
Name: lr_3w0, dtype: float64
ml_3w0
count    1.112344e+06
mean     3.698418e+00
std      4.892294e+00
min      3.548020e-04
25%      1.080575e+00
50%      2.188350e+00
75%      4.696698e+00
max      1.460685e+02
Name: ml_3w0, dtype: float64
as_3w0
count    1.112344e+06
mean     4.179343e+00
std      5.654623e+00
min      1.177057e-03
25%      1.175206e+00
50%      2.509872e+00
75%      5.210507e+00
max      1.987954e+02
Name: as_3w0, dtype: float64
bl_3w5
count    1.112344e+06
mean     3.073298e+01
std      2.486576e+01
min      3.219377e-03
25%      8.615625e+00
50%      2.393974e+01
75%      5.01